![CoSAppLogo](../images/cosapp.svg)

<font color='orange'>**CoSApp**</font> tutorials on multimode systems

# Simple Newton Pendulum

In [ ]:
from cosapp.base import System
import numpy as np


class Pendulum(System):
    """Point mass planar pendulum model"""
    def setup(self):
        self.add_inward('L', 1.00, desc='Length of the rod')
        self.add_inward('g', 9.81, desc='Gravitational acceleration')
        
        self.add_outward('acc', 0., desc="Angular acceleration of the mass")
        self.add_transient('omega', der='acc', desc="Angular velocity")
        self.add_transient('theta', der='omega', desc="Positional angle")

    def compute(self):
        self.acc = -(self.g / self.L) * np.sin(self.theta)


class NewtonPendulum(System):
    def setup(self):
        self.add_child(Pendulum('p1'), pulling=['L', 'g'])
        self.add_child(Pendulum('p2'), pulling=['L', 'g'])
        
        contact = self.add_event('contact', trigger="p1.theta == p2.theta")
        self.add_event('collision', trigger=contact.filter("p1.omega > p2.omega"))

    def transition(self):
        if self.collision.present:
            p1, p2 = self.p1, self.p2
            # swap angular velocities
            p1.omega, p2.omega = p2.omega, p1.omega


In [ ]:
from cosapp.drivers import RungeKutta
from cosapp.recorders import DataFrameRecorder

pend = NewtonPendulum("pend")

driver = pend.add_driver(
    RungeKutta(order=3, dt=0.01, time_interval=[0, 10])
)
driver.set_scenario(
    init = {
        "p1.theta" : np.radians(-60),
        "p1.omega" : 2,
        "p2.theta" : np.radians(22),
        "p2.omega" : 0.5,
    },
    values = {
        "L" : 0.5,
        "g" : 9.81,
    },
    # stop = pend.collision.filter('p1.theta < 0'),  # try it!
)

# Recorder
driver.add_recorder(
    DataFrameRecorder(includes=[
        'p?.theta', 'p?.omega',
    ]),
    period = 0.05,
)

# Run simulation & retrieve recorded data
pend.run_drivers()
data = driver.recorder.export_data()
data = data.drop(['Section', 'Status', 'Error code'], axis=1)

In [ ]:
# Plot results
import plotly.graph_objs as go

traces = [
    go.Scatter(
        x = data['time'],
        y = data['p1.theta'],
        mode='lines', name='Pendulum 1', line=dict(color='red'),
    ),
    go.Scatter(
        x = data['time'],
        y = data['p2.theta'],
        mode='lines', name='Pendulum 2', line=dict(color='blue'),
    ),
]
layout = go.Layout(
    title = "Angular positions",
    xaxis = dict(title="Time"),
    height = 450,
    hovermode = "x",
)
go.Figure(data=traces, layout=layout)

Time driver property `recorded_events` contains a list of all recorded events.
Each element of the list is a named tuple `EventRecord(time, events)`, containing the occurrence time, and the list of cascading events at that time, starting by the primitive event.

In [ ]:
driver.recorded_events

Time driver property `event_data` contains the same fields as the recorder, but only at event times:

In [ ]:
driver.event_data.drop(['Section', 'Status', 'Error code'], axis=1)